In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.mida import Mida
from models.mice import Mice

import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from fancyimpute import KNN, SoftImpute

from preprocessing.data_loading import DataSet
from preprocessing.missing_value_generation import mcar_generator, mar_generator, mnar_generator
from models.imputation_wrapper import SingleImputationWrapper, MultiImputationWrapper
from analysis.evaluation import ClassificationEvaluation

/home/marcus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
dataset = DataSet('ionosphere', 'class')
amputation = mcar_generator()
dataset.init_missing_data(amputation)

In [4]:
clf_evaluation = ClassificationEvaluation(KNeighborsClassifier(5), dataset)

In [5]:
# result on complete dataset
clf_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')

In [6]:
imputations = [SingleImputationWrapper(KNN(3)), SingleImputationWrapper(SoftImpute()),
               MultiImputationWrapper(Mice(10))]

In [7]:
#increase missing data rate
for p in [0.2, 0.4, 0.6]:
    dataset.ampute_values(p)
    
    # apply each imputation
    for imputation in imputations:
        results = imputation.complete(dataset.missing_data())
        clf_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())

/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan


Imputing row 1/351 with 1 missing, elapsed time: 0.044
Imputing row 101/351 with 1 missing, elapsed time: 0.047
Imputing row 201/351 with 1 missing, elapsed time: 0.050
Imputing row 301/351 with 2 missing, elapsed time: 0.052
[SoftImpute] Max Singular Value of X_init = 45.127243
[SoftImpute] Iter 1: observed MAE=0.032478 rank=33
[SoftImpute] Iter 2: observed MAE=0.032490 rank=33
[SoftImpute] Iter 3: observed MAE=0.032500 rank=33
[SoftImpute] Iter 4: observed MAE=0.032507 rank=33
[SoftImpute] Iter 5: observed MAE=0.032512 rank=33
[SoftImpute] Iter 6: observed MAE=0.032515 rank=33
[SoftImpute] Iter 7: observed MAE=0.032516 rank=33
[SoftImpute] Iter 8: observed MAE=0.032515 rank=33
[SoftImpute] Iter 9: observed MAE=0.032514 rank=33
[SoftImpute] Iter 10: observed MAE=0.032512 rank=33
[SoftImpute] Iter 11: observed MAE=0.032509 rank=33
[SoftImpute] Iter 12: observed MAE=0.032505 rank=33
[SoftImpute] Iter 13: observed MAE=0.032502 rank=33
[SoftImpute] Iter 14: observed MAE=0.032498 rank=33
[

/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars



[SoftImpute] Iter 43: observed MAE=0.032436 rank=33
[SoftImpute] Iter 44: observed MAE=0.032436 rank=33
[SoftImpute] Iter 45: observed MAE=0.032435 rank=33
[SoftImpute] Iter 46: observed MAE=0.032435 rank=33
[SoftImpute] Iter 47: observed MAE=0.032435 rank=33
[SoftImpute] Iter 48: observed MAE=0.032434 rank=33
[SoftImpute] Iter 49: observed MAE=0.032434 rank=33
[SoftImpute] Iter 50: observed MAE=0.032434 rank=33
[SoftImpute] Iter 51: observed MAE=0.032434 rank=33
[SoftImpute] Stopped after iteration 51 for lambda=0.902545
[MICE] Completing matrix with shape (351, 34)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.005
[MICE] Starting imputation round 3/11, elapsed time 0.021
[MICE] Starting imputation round 4/11, elapsed time 0.028
[MICE] Starting imputation round 5/11, elapsed time 0.035
[MICE] Starting imputation round 6/11, elapsed time 0.042
[MICE] Starting imputation round 7/11, elapsed time 0.049
[MICE] Starting imp

/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan


Imputing row 1/351 with 2 missing, elapsed time: 0.051
Imputing row 101/351 with 1 missing, elapsed time: 0.055
Imputing row 201/351 with 2 missing, elapsed time: 0.058
Imputing row 301/351 with 2 missing, elapsed time: 0.062
[SoftImpute] Max Singular Value of X_init = 44.112469
[SoftImpute] Iter 1: observed MAE=0.032245 rank=33
[SoftImpute] Iter 2: observed MAE=0.032255 rank=33
[SoftImpute] Iter 3: observed MAE=0.032263 rank=33
[SoftImpute] Iter 4: observed MAE=0.032269 rank=33
[SoftImpute] Iter 5: observed MAE=0.032273 rank=33
[SoftImpute] Iter 6: observed MAE=0.032276 rank=33
[SoftImpute] Iter 7: observed MAE=0.032278 rank=33
[SoftImpute] Iter 8: observed MAE=0.032279 rank=33
[SoftImpute] Iter 9: observed MAE=0.032279 rank=33
[SoftImpute] Iter 10: observed MAE=0.032277 rank=33
[SoftImpute] Iter 11: observed MAE=0.032274 rank=33
[SoftImpute] Iter 12: observed MAE=0.032269 rank=33
[SoftImpute] Iter 13: observed MAE=0.032264 rank=33
[SoftImpute] Iter 14: observed MAE=0.032259 rank=33
[

/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


[SoftImpute] Iter 56: observed MAE=0.032121 rank=33
[SoftImpute] Iter 57: observed MAE=0.032121 rank=33
[SoftImpute] Iter 58: observed MAE=0.032120 rank=33
[SoftImpute] Iter 59: observed MAE=0.032120 rank=33
[SoftImpute] Iter 60: observed MAE=0.032119 rank=33
[SoftImpute] Iter 61: observed MAE=0.032119 rank=33
[SoftImpute] Iter 62: observed MAE=0.032118 rank=33
[SoftImpute] Stopped after iteration 62 for lambda=0.882249
[MICE] Completing matrix with shape (351, 34)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.015
[MICE] Starting imputation round 3/11, elapsed time 0.021
[MICE] Starting imputation round 4/11, elapsed time 0.024
[MICE] Starting imputation round 5/11, elapsed time 0.027
[MICE] Starting imputation round 6/11, elapsed time 0.029
[MICE] Starting imputation round 7/11, elapsed time 0.032
[MICE] Starting imputation round 8/11, elapsed time 0.035
[MICE] Starting imputation round 9/11, elapsed time 0.038
[MICE] S

[SoftImpute] Iter 26: observed MAE=0.032200 rank=33
[SoftImpute] Iter 27: observed MAE=0.032196 rank=33
[SoftImpute] Iter 28: observed MAE=0.032192 rank=33
[SoftImpute] Iter 29: observed MAE=0.032187 rank=33
[SoftImpute] Iter 30: observed MAE=0.032183 rank=33
[SoftImpute] Iter 31: observed MAE=0.032179 rank=33
[SoftImpute] Iter 32: observed MAE=0.032174 rank=33
[SoftImpute] Iter 33: observed MAE=0.032170 rank=33
[SoftImpute] Iter 34: observed MAE=0.032166 rank=33
[SoftImpute] Iter 35: observed MAE=0.032162 rank=33
[SoftImpute] Iter 36: observed MAE=0.032158 rank=33
[SoftImpute] Iter 37: observed MAE=0.032154 rank=33
[SoftImpute] Iter 38: observed MAE=0.032151 rank=33
[SoftImpute] Iter 39: observed MAE=0.032148 rank=33
[SoftImpute] Iter 40: observed MAE=0.032144 rank=33
[SoftImpute] Iter 41: observed MAE=0.032141 rank=33
[SoftImpute] Iter 42: observed MAE=0.032138 rank=33
[SoftImpute] Iter 43: observed MAE=0.032136 rank=33
[SoftImpute] Iter 44: observed MAE=0.032133 rank=33
[SoftImpute]

[MICE] Starting imputation round 11/11, elapsed time 0.141
[MICE] Completing matrix with shape (351, 34)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.006
[MICE] Starting imputation round 3/11, elapsed time 0.012
[MICE] Starting imputation round 4/11, elapsed time 0.016
[MICE] Starting imputation round 5/11, elapsed time 0.020
[MICE] Starting imputation round 6/11, elapsed time 0.023
[MICE] Starting imputation round 7/11, elapsed time 0.026
[MICE] Starting imputation round 8/11, elapsed time 0.029
[MICE] Starting imputation round 9/11, elapsed time 0.032
[MICE] Starting imputation round 10/11, elapsed time 0.035
[MICE] Starting imputation round 11/11, elapsed time 0.039
[MICE] Completing matrix with shape (351, 34)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.004
[MICE] Starting imputation round 3/11, elapsed time 0.007
[MICE] Starting imputation round 4/

In [8]:
clf_evaluation.evaluation_results

,"(settings, p)","(settings, mechanism)","(settings, imputation)","(settings, number)","(accuracy, post)","(accuracy, prev)"
0,0,None,None,1,0.768293,0.768293
1,0.2,mcar,KNN,1,0.764228,0.764228
2,0.2,mcar,SoftImpute,1,0.752033,0.752033
3,0.2,mcar,Mice,1,0.780488,0.780488
4,0.2,mcar,Mice,5,0.780488,0.764228
5,0.2,mcar,Mice,10,0.776423,0.764228
6,0.4,mcar,KNN,1,0.780488,0.780488
7,0.4,mcar,SoftImpute,1,0.719512,0.719512
8,0.4,mcar,Mice,1,0.723577,0.723577
9,0.4,mcar,Mice,5,0.739837,0.752033
